In [ ]:
import pandas as pd
import re
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from google.cloud import translate_v2 as translate
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key'

In [ ]:
# 원본 파일 읽어옴
df = pd.read_csv('./overwatch_2022-01-01_2022-05-05tweetdata.csv', index_col=0)

In [ ]:
emoj = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002500-\U00002BEF"  # chinese char
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    u"\U0001f926-\U0001f937"
    u"\U00010000-\U0010ffff"
    u"\u2640-\u2642" 
    u"\u2600-\u2B55"
    u"\u200d"
    u"\u23cf"
    u"\u23e9"
    u"\u231a"
    u"\ufe0f"  # dingbats
    u"\u3030"
                     "]+", re.UNICODE)

def preprocess(str):
    # 불필요한 공백 삭제
    str = str.strip(" ")
    
    # @이나 RT로 시작하는 부분은 필요없다고 가정하고 모두 삭제
    while(1):
        if(str[0:1] == '@' or str[0:2] == 'RT') :
            str = ' '.join(str.split()[1:])
        else : break
    
    # 이모지 지움
    str = re.sub(emoj, '', str)
    
    # 특수문자, https 지움
    str = re.sub('#|@|\*|,|http.*|\^', '', str)
    
    # 공백 문자 실제 공백으로 대체
    str = re.sub('\s|-|:|;|&|/|\(|\)|\"|“|”|\[|\]', ' ', str)
    
    # 물음표, 느낌표 온점으로 대체
    str = re.sub('!|\?', '.', str)
    str = re.sub('’|‘', '\'', str)
    
    # 대문자 소문자로 바꿈
    str = str.lower()
    
    # 다시 불필요한 공백 제거
    str = str.strip(" ")
    
    return str

In [ ]:
preprocessed = []

client = translate.Client()

for i, s in enumerate(df['Text']):
    temp = preprocess(s)
   
    # 길이가 2보다 작은 경우엔 무의미하다고 생각
    if(len(temp) < 2) : continue
    # 영어인 경우에만 저장
    if client.detect_language(temp)['language'] == 'en':
        preprocessed.append(temp)

In [ ]:
tokenizer = TreebankWordTokenizer()
stop_words = set(stopwords.words('english')) 

In [ ]:
for i, s in enumerate(preprocessed):
    tokens = tokenizer.tokenize(s)
    temp = []
    
    for j in tokens:
        if j not in stop_words :
            temp.append(j)

    preprocessed[i] = ' '.join(temp)

In [ ]:
col = ['Preprocessed']
result = pd.DataFrame(preprocessed, columns=col)
result

In [ ]:
result.to_csv('result.csv')